In [7]:
import numpy as np
import pandas as pd
import datetime
import warnings
import openpyxl
warnings.filterwarnings("ignore")

# Lesion(TRGOC)가 있는데, TRGOC, TRGOCOT가 둘 다 없는 경우
# Lesion(TRGOC)가 있는데, TRGOC 또는 TRGOCOT가  없는 경우

In [13]:
#kwargs 의 value값은 and, or로 지정, ex) operator = and

def andor(range1, range2, dataframe,*args, **kwargs):
     
    #df_TL에 DM_CMT 컬럼 추가
    new_list = list(dataframe.columns)
    new_list.append("DM_CMT")
    
    #col1 = string
    #col2 = string

    #범위 지정->string으로 받는다
    list_range = [str(i) for i in list(range(range1, range2))]
    
    #kwargs의 key값, value값을 리스트로 받고 인덱싱 한다 -> 나오는 결과괎: ex."TRGMET"
    key = [keys for keys in kwargs.keys()][0]
    value = [values for values in kwargs.values()][0]
    
    #변수가 3개일때
    #operator가 and 일때 , -> TRGOCOT도 없고, TRGOCOSIT도 없는경우
    if len(args)==3:
        if value=="and":
            #빈 데이터프레임 생성
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)
            for numlist in list_range:
                df_empty = dataframe[(dataframe[args[0]+"_"+numlist].notnull()) & ( (dataframe[args[1]+"_"+numlist].isnull()) & (dataframe[args[2]+"_"+numlist].isnull()) )]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 value가 있는데, "+args[1]+"_"+numlist+"이 없고,"+args[2]+"_"+numlist+"도 없는 경우"
                df_append = df_append.append(df_empty)
                
        if value=="or":
            #빈 데이터프레임 생성
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)
            for numlist in list_range:
                df_empty = dataframe[(dataframe[args[0]+"_"+numlist].isnull()) & ( (dataframe[args[1]+"_"+numlist].notnull()) | (dataframe[args[2]+"_"+numlist].notnull()) )]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 NA인데, "+args[1]+"_"+numlist+"이 있거나,"+args[2]+"_"+numlist+"이 있는 경우"
                df_append = df_append.append(df_empty)
    
    
    

    return df_append
    

# NA, Value_v2

In [56]:
def valuena(range1, range2, dataframe, *args, **kwargs):
    
    #df_TL에 DM_CMT 컬럼 추가
    new_list = list(dataframe.columns)
    new_list.append("DM_CMT")

    #범위 지정->string으로 받는다
    list_range = [str(i) for i in list(range(range1, range2))]
    
    #kwargs의 key값, value값을 리스트로 받고 인덱싱 한다 -> 나오는 결과괎: ex."TRGMET"
    key = [keys for keys in kwargs.keys()]
    value = [values for values in kwargs.values()]
    
    #TRGOC값이 TRGLD가 없는 경우
    #response 중 NE, CR이 포함된 경우는 제외
    #valuena(1,6,df_TL,"TRGOC","TRGLD", response="TRGRESP",exclude=["NE","CR"])
    if "response" in key:
        #빈 데이터프레임 생성
        df_empty = pd.DataFrame( columns = new_list)
        df_append = pd.DataFrame( columns = new_list)

        for numlist in list_range:
            df_empty = dataframe[(dataframe[args[0]+"_"+numlist].notnull())  &  (dataframe[args[1]+"_"+numlist].isnull())]
            df_empty = df_empty[-df_empty[kwargs["response"]].isin(kwargs["exclude"])]
            df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 value가 있는데,"+args[1]+numlist+"가 na인 경우"
            df_append = df_append.append(df_empty)
    
    
    #tumor길이에 따라 달라지는 경우
    if "length" in key:
        #튜머길이가 없을 경우        
        if kwargs["length"] == "NA":
            #빈 데이터프레임 생성
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)

            for numlist in list_range:
                df_empty = dataframe[dataframe[args[0]+"_"+numlist].notnull()  &  dataframe[args[1]+"_"+numlist].isnull()]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 value가 있는데,"+args[1]+numlist+"가 na인 경우"
                df_append = df_append.append(df_empty)
                

        
        #튜머길이가 있을 경우
        if kwargs["length"] != "NA":
            #빈 데이터프레임 생성
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)

            for numlist in list_range:
                df_empty = dataframe[dataframe[args[0]+"_"+numlist].notnull()  &  dataframe[args[1]+"_"+numlist].isnull()]
                df_empty = df_empty[df_empty[kwargs["length"]+"_"+numlist]!=0]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 value가 있는데,"+args[1]+"_"+numlist+"가 na인 경우"
                df_append = df_append.append(df_empty)
        
    return df_append





#lenth = tumor 길이
# lenght = "NA" -> 매개변수 기본값 설정 -> 함수에 lenght값을 기입하지 않으면 기본값(여기에서는 NA)값이 기입됨
# length 기입 X -> 매칭이 안됨 -> 기본값 NA를 가지고 있기 때문에 NA로 기입됨 -> tumor길이를 고려하지 않고 함수적용
# length 기입 O -> 매칭        -> 기입한 컬럼으로 기입됨 -> tumor길이가 0이 아닌 경우 query내용 보여줌

def navalue(range1, range2, dataframe, col1, col2, length="NA"):
    
    #df_TL에 DM_CMT 컬럼 추가
    new_list = list(dataframe.columns)
    new_list.append("DM_CMT")
    
    #빈 데이터프레임 생성
    df_empty = pd.DataFrame( columns = new_list)
 
    #범위 지정->string으로 받는다
    list_range = [str(i) for i in list(range(range1, range2))]
    
    if length == "NA":
        #빈 데이터프레임 생성
        df_empty = pd.DataFrame( columns = new_list)
        df_append = pd.DataFrame( columns = new_list)
        for numlist in list_range:
            df_empty = dataframe[dataframe[col1+"_"+numlist].isnull()  &  dataframe[col2+"_"+numlist].notnull()]
            df_empty["DM_CMT"] = col1+"_"+numlist+"가 na이지만,"+col2+numlist+"가 value가 있는 경우"
            df_append = df_append.append(df_empty)
            
    if length != "NA":
        #빈 데이터프레임 생성
        df_empty = pd.DataFrame( columns = new_list)
        df_append = pd.DataFrame( columns = new_list)
        for numlist in list_range:
            df_empty = dataframe[dataframe[col1+"_"+numlist].isnull()  &  dataframe[col2+"_"+numlist].notnull()]
            df_empty = df_empty[df_empty[length+"_"+numlist]!=0]
            df_empty["DM_CMT"] = col1+"_"+numlist+"가 na이지만,"+col2+numlist+"가 value가 있는 경우"
            df_append = df_append.append(df_empty)
             
    return df_append

# NA, Value TRGDL_SE, TRGDL_IM

In [9]:
def valuenaseim(range1, range2, dataframe, *args, **kwargs):
    
    #df_TL에 DM_CMT 컬럼 추가
    new_list = list(dataframe.columns)
    new_list.append("DM_CMT")

    #범위 지정->string으로 받는다
    list_range = [str(i) for i in list(range(range1, range2))]
    
    #kwargs의 key값, value값을 리스트로 받고 인덱싱 한다 -> 나오는 결과괎: ex."TRGMET"
    key = [keys for keys in kwargs.keys()]
    value = [values for values in kwargs.values()]

    #col2 가 TRGDL_SE일 때
    #length=0, Response가 "NE","CR"이면 제외
    if "TRGDL_SE" in args:
        #튜머길이가 있을 경우, 
        #Response가 CR 또는 NE일경우 제외
        if kwargs["length"] != "NA":
            if "response" in key:
                #빈 데이터프레임 생성
                df_empty = pd.DataFrame( columns = new_list)
                df_append = pd.DataFrame( columns = new_list)

                for numlist in list_range:
                    df_empty = dataframe[(dataframe[args[0]+"_"+numlist].notnull())  &  (dataframe[args[1]+numlist].isnull())]
                    df_empty = df_empty[df_empty[kwargs["length"]+"_"+numlist]!=0]
                    df_empty = df_empty[-df_empty[kwargs["response"]].isin(kwargs["exclude"])]
                    df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 value가 있는데,"+args[1]+numlist+"가 na인 경우"
                    df_append = df_append.append(df_empty)


    #col2 가 TRGDL_IM일 때
    if "TRGDL_IM" in args:
        #튜머길이가 있을 경우, 
        #Response가 CR 또는 NE일경우 제외
        if kwargs["length"] != "NA":
            if "response" in key:
                #빈 데이터프레임 생성
                df_empty = pd.DataFrame( columns = new_list)
                df_append = pd.DataFrame( columns = new_list)

                for numlist in list_range:
                    df_empty = dataframe[dataframe[args[0]+"_"+numlist].notnull()  &  dataframe[args[1]+numlist].isnull()]
                    df_empty = df_empty[df_empty[kwargs["length"]+"_"+numlist]!=0]
                    df_empty = df_empty[-df_empty[kwargs["response"]].isin(kwargs["exclude"])]
                    df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 value가 있는데,"+args[1]+numlist+"가 na인 경우"
                    df_append = df_append.append(df_empty)
                    
    return df_append

# Other, Na, value

In [8]:
#TRGMET 값이 Other인데 , TRGMETOT_n 이 없는 경우(TRGLD_n이 0이 아닌경우)

def otherna(range1, range2, dataframe,*args, **kwargs):
    
    #df_TL에 DM_CMT 컬럼 추가(df_TL에는 DM_CMT컬럼이 없기 때문에)
    new_list = list(dataframe.columns)
    new_list.append("DM_CMT")    
    
    #범위 지정->string으로 받는다
    list_range = [str(i) for i in list(range(range1, range2))]
    
    #kwargs의 key값, value값을 리스트로 받고 인덱싱 한다 -> 나오는 결과괎: ex."TRGMET"
    key = [keys for keys in kwargs.keys()][0]
    value = [values for values in kwargs.values()][0]
    
    
    #변수가 3개일때
    #tumor길이가 필요없을 때, 일반적인 상황 : if value=="NA" ->default값
    #ex) otherna(1,6,df_TL,"TRGOC","TRGOCOT","TRGOCSITE",length="NA")
    if len(args)==3:
        if value=="NA":
            #빈 데이터프레임 생성
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)
            for numlist in list_range:
                df_empty = dataframe[(dataframe[args[0]+"_"+numlist]=="Other") & ( (dataframe[args[1]+"_"+numlist].isnull()) & (dataframe[args[2]+"_"+numlist].isnull()) )]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"가 Other인데 "+args[1]+"_"+numlist+"이 없고,"+args[2]+"_"+numlist+"도 없는 경우"
                df_append = df_append.append(df_empty)
                
                
    #변수가 2개일때
    #tumor길이가 필요할 때, 특수한 상황 : if value!="NA" ->tumor길이가 0이 아닌 경우
    #ex) otherna(1,6,df_TL,"TRGMET","TRGMETOT",length="TRGLD")
    if len(args)==2:
        if value!="NA":
            #빈 데이터프레임 생성
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)
            for numlist in list_range:
                #결과가 출력될 dataframe = df_NA : TRGOCSITE_1_NA
                df_empty = dataframe[(dataframe[args[0]+"_"+numlist]=="Other")  & ( dataframe[args[1]+"_"+numlist].isnull() )]

                #value값(tumor길이)가 0인것은 제외 
                df_empty = df_empty[df_empty[value+"_"+numlist]!=0]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"값이 Other인데,"+args[1]+numlist+"가 없는 경우"
                df_append = df_append.append(df_empty)
    
    
    #최종 데이터프레임 산출
    return df_append




def nothervalue(range1, range2, dataframe,*args, **kwargs):
    
    #df_TL에 DM_CMT 컬럼 추가(df_TL에는 DM_CMT컬럼이 없기 때문에)
    new_list = list(dataframe.columns)
    new_list.append("DM_CMT")
    
    #범위 지정->string으로 받는다
    list_range = [str(i) for i in list(range(range1, range2))]
    
    #kwargs의 key값, value값을 리스트로 받고 인덱싱 한다 -> 나오는 결과괎: ex."TRGMET"
    key = [keys for keys in kwargs.keys()][0]
    value = [values for values in kwargs.values()][0]
                      
    #변수가 2개일때
        #tumor길이가 필요할 때, 일반적인 상황 : if value!="NA":
    if len(args)==2:
        if value=="NA":
            df_empty = pd.DataFrame( columns = new_list)
            df_append = pd.DataFrame( columns = new_list)
            for numlist in list_range:
                df_empty = dataframe[(dataframe[args[0]+"_"+numlist]!="Other")  & ( dataframe[args[1]+"_"+numlist].notnull() )]
                df_empty["DM_CMT"] = args[0]+"_"+numlist+"값이 Other가 아닌데,"+args[1]+numlist+"가 있는 경우"
                df_append = df_append.append(df_empty)
    
    #최종 데이터프레임 산출
    return df_append